In [36]:
import islpy as isl
from src.io_tools import *
from src.graph_tools import *
from src.swap_tools import *
from prettytable import PrettyTable

In [37]:
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger('stevedore.extension').setLevel(logging.CRITICAL)


In [38]:
json_file_path = 'benchmarks/polyhedral/cases/bigd.json'

data = json_file_to_isl(json_file_path)

In [39]:

domain, access, schedule = filter_multi_qubit_gates(data["domain"], data["read_dependencies"], data["schedule"])


In [40]:
regex = re.compile(r'(?<=\[)(.*?)(?=\])')
num_qubits = int(regex.findall(data['read_dependencies'].range().as_set().lexmax().to_str())[0]) + 1

In [41]:
physical_qubits_domain = isl.Set(f"{{ [i] : 1 <= i <= {num_qubits} }}")

In [42]:
# Generate a 2D grid for 200 qubits (10 rows and 20 columns)
def generate_2d_grid(num_rows, num_cols):
    num_qubits = num_rows * num_cols
    graph = [[] for _ in range(num_qubits)]
    
    for i in range(num_rows):
        for j in range(num_cols):
            index = i * num_cols + j
            # Connect to the left neighbor
            if j > 0:
                graph[index].append(index - 1)
            # Connect to the right neighbor
            if j + 1 < num_cols:
                graph[index].append(index + 1)
            # Connect to the top neighbor
            if i > 0:
                graph[index].append(index - num_cols)
            # Connect to the bottom neighbor
            if i + 1 < num_rows:
                graph[index].append(index + num_cols)
    
    return graph

num_rows = 4
num_cols = 5
graph = generate_2d_grid(num_rows, num_cols)



In [43]:
backend_disconnected_edges =  extract_edges_map(graph)
shortest_paths = extract_shortest_paths(graph)

In [44]:
initial_mapping = isl.Map(f"{{ q[i] -> [i] : 1<=i<={num_qubits} }}")

In [45]:
mapping = initial_mapping
swap_count = 0
programme = schedule
intersect_set = isl.Set("{ [i, j] : i > j }")

In [46]:
import time

In [47]:
iteration = 0
start = time.time()
while not programme.is_empty():
    #print(f"Iteration {iteration}")
    #print(programme.domain())
    iteration += 1
    programme_mapped = programme.apply_range(mapping)
    programme_access = programme_mapped.flat_range_product(programme_mapped).intersect_range(intersect_set)
    first_disconnected_edge = programme_access.intersect_range(backend_disconnected_edges).domain().lexmin()
    if first_disconnected_edge.is_empty():
        break

    disconected_qubits = programme_mapped.intersect_domain(first_disconnected_edge).range().as_set()
    q1 , q2 = disconected_qubits.dim_min_val(0).to_python(),disconected_qubits.dim_max_val(0).to_python()
    swap_count += shortest_paths[q1]['costs'][q2] -1
    new_domain = programme.domain().as_set().lex_gt_set(first_disconnected_edge.as_set()).domain()
    programme = programme.intersect_domain(new_domain).coalesce()
    swap_map = swaps_to_isl_map(shortest_paths[q1]['paths'][q2])
    mapping = apply_swaps_to_logical_qubits_map(swap_map,mapping,physical_qubits_domain)

end = time.time()


In [48]:
swap_count

12

In [49]:
iteration

5

In [50]:
results_table = PrettyTable()

results_table.field_names = ["Method","Swap Count", "Execution Time (s)"]
results_table.add_row(["Our solution", swap_count, end-start])

----------------------

In [51]:
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import CouplingMap, PassManager ,Layout
from qiskit.transpiler.passes import LookaheadSwap, SabreSwap
from qiskit_aer import Aer

In [52]:
qasm_code = json_file_qasm(json_file_path)
qc = QuantumCircuit.from_qasm_str(qasm_code)

num_qubits = qc.num_qubits

In [53]:

#qc.draw(output='mpl')

In [54]:
edges = [(i, j) for i, neighbors in enumerate(graph) for j in neighbors if i < j]


coupling_map = CouplingMap(edges)

In [55]:
layout_dict = {qc.qubits[i]: i for i in range(num_qubits)}  
layout = Layout(layout_dict)

In [56]:
pass_manager = PassManager()
pass_manager.append(LookaheadSwap(coupling_map))

In [57]:
qc.count_ops()

OrderedDict([('x', 15), ('cx', 5)])

In [58]:
start_time = time.time()
transpiled_circuit = transpile(qc, coupling_map=coupling_map, initial_layout=layout)
optimized_circuit = pass_manager.run(transpiled_circuit)
end_time = time.time()

In [59]:
swap_count = sum(1 for op in transpiled_circuit.data if op[0].name == 'swap')

In [60]:
results_table.add_row(["LookaheadSwap", swap_count, end_time-start_time])

In [61]:
pass_manager = PassManager()
pass_manager.append(SabreSwap(coupling_map))

start_time = time.time()
transpiled_circuit = transpile(qc, coupling_map=coupling_map, initial_layout=layout)
optimized_circuit = pass_manager.run(transpiled_circuit)
end_time = time.time()

swap_count = sum(1 for op in transpiled_circuit.data if op[0].name == 'swap')

results_table.add_row(["SabreSwap", swap_count, end_time-start_time])

In [62]:
start_time = time.time()
transpiled_circuit = transpile(qc, coupling_map=coupling_map, initial_layout=layout,optimization_level=0)
end_time = time.time()
swap_count = sum(1 for op in transpiled_circuit.data if op[0].name == 'swap')

results_table.add_row(["without optimizer", swap_count, end_time-start_time])

In [63]:
print(results_table)

+-------------------+------------+----------------------+
|       Method      | Swap Count |  Execution Time (s)  |
+-------------------+------------+----------------------+
|    Our solution   |     12     | 0.036759138107299805 |
|   LookaheadSwap   |     13     |  1.8237488269805908  |
|     SabreSwap     |     13     | 0.10666656494140625  |
| without optimizer |     12     | 0.06656193733215332  |
+-------------------+------------+----------------------+
